In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.autograd import Variable

In [3]:
train_dataset = datasets.MNIST(root="./data",
                              train=True,
                              transform=transforms.ToTensor(),
                              download=True)

Processing...
Done!


In [4]:
test_dataset = datasets.MNIST(root="./data",
                              train=False,
                              transform=transforms.ToTensor(),
                              )

In [7]:
batch_size = 100
n_iters = 3000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

In [8]:
num_epochs

5

In [9]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

In [47]:
class NeuralNetworkModel(nn.Module):
    def __init__(self, input_size, hidden_dim, output_dim):
        super(NeuralNetworkModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.sigmoid = nn.Sigmoid()
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        output = self.fc1(x)
        
        output = self.sigmoid(output)
        
        output = self.fc2(output)
        
        return output
    

In [48]:
input_dim = 28 * 28
hidden_dim = 100
output_dim = 10

In [49]:
model = NeuralNetworkModel(input_dim, hidden_dim, output_dim)

In [50]:
criterion = nn.CrossEntropyLoss()

In [51]:
learning_rate = 0.1

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [52]:
print(model.parameters())

print(len(list(model.parameters())))

print(list(model.parameters())[0].size())

print(list(model.parameters())[1].size())

print(list(model.parameters())[2].size())

print(list(model.parameters())[3].size())

<generator object Module.parameters at 0x1182da9e8>
4
torch.Size([100, 784])
torch.Size([100])
torch.Size([10, 100])
torch.Size([10])


In [54]:
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1, 28 * 28))
        labels = Variable(labels)
        
        optimizer.zero_grad()
        
        outputs = model(images)
        
        loss = criterion(outputs, labels)
        
        loss.backward()
        
        optimizer.step()
        
        iter += 1
        
        if iter % 500 == 0:
            correct = 0
            total = 0
            
            for images, labels in test_loader:
                images = Variable(images.view(-1, 28*28))
                lables = Variable(labels)
                
                outputs = model(images)
                
                _, predicted = torch.max(outputs.data, 1)
                
                total += labels.size(0)
                
                correct += (predicted == labels).sum()
                
            accuracy = 100 * correct /total
            
            print("Iteration {}, loss: {}, accuracy: {}".format(iter, loss.data[0], accuracy))

Iteration 500, loss: 0.5759841799736023, accuracy: 86.07
Iteration 1000, loss: 0.3442213833332062, accuracy: 89.41
Iteration 1500, loss: 0.2572471499443054, accuracy: 90.51
Iteration 2000, loss: 0.49963080883026123, accuracy: 91.3
Iteration 2500, loss: 0.31837591528892517, accuracy: 91.55
Iteration 3000, loss: 0.3280375003814697, accuracy: 92.14


In [55]:
torch.save(model.state_dict(), 'neuralnet_model.pkl')